# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [69]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Get all the source files

In [70]:
# Check the current working directory
print(os.getcwd())

# Collect all the files inside the 'event_data' folder
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):    
    file_path_list = glob.glob(os.path.join(root,'*'))
    files = pd.DataFrame(file_path_list, columns=['File Uri'])
    files = files.rename(index = lambda x: x + 1)
    
files.style

/workspace/home


,File Uri
1,/workspace/home/event_data/2018-11-10-events.csv
2,/workspace/home/event_data/2018-11-20-events.csv
3,/workspace/home/event_data/2018-11-09-events.csv
4,/workspace/home/event_data/2018-11-11-events.csv
5,/workspace/home/event_data/2018-11-07-events.csv
6,/workspace/home/event_data/2018-11-27-events.csv
7,/workspace/home/event_data/2018-11-21-events.csv
8,/workspace/home/event_data/2018-11-15-events.csv
9,/workspace/home/event_data/2018-11-17-events.csv
10,/workspace/home/event_data/2018-11-06-events.csv


#### Pre-processing the files to create the data file csv that will be used for Apache Casssandra tables

In [71]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# Read each line in each file and append them 
# in the emplty list initialised above
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)       
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# Collect the data we are interested into a new file called 'event_datafile_new.csv'
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    # write the column headers as the first row of the CSV
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    # write the rest of the data line by line
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
print('DONE!')

DONE!


In [72]:
# check the number of rows the new file contains
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))


6821


# Part II. Extracting the data prepared above into Cassandra database. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing the data into the Apache Cassandra

In [73]:
# To establish a connection session to a local Cassandra database cluster
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print('Connected!')
except Exception as e:
    print(e)

Connected!


#### Create Keyspace

In [74]:
# Create a Keyspace called 'project1'
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project1 WITH REPLICATION = { 'class': 'SimpleStrategy', 'replication_factor' : 1}
    """)
    print('Keyspace Created!')
except Exception as e:
    print(e)

Keyspace Created!


#### Set Keyspace

In [75]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project1')
    print('Session SET!')
except Exception as e:
    print(e)

Session SET!


## We will model our data and create three tables that will be optimised for each query.

### Query 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### To satisfy the above query, we will need the sessionId column, itemInSession column, artist column, the song column and the length column.  The primary key of the table are sessionId and itemInSession, since the two columns on their own separately are not sufficient to uniquely identify a table row.  However, together as a composite partition key, they seem to be sufficient

In [76]:
query1 = """
CREATE TABLE IF NOT EXISTS app_history 
(sessionId INT, itemInSession INT, artist TEXT, song TEXT, length FLOAT, 
PRIMARY KEY(sessionId, itemInSession));
"""

try:
    session.execute('DROP TABLE IF EXISTS app_history;')
    session.execute(query1)
    print('app_history CREATED!')
except Exception as e:
    print(e)
                    

app_history CREATED!


In [77]:
# Populate the table with data

file = 'event_datafile_new.csv'

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            query = "INSERT INTO app_history (sessionId, itemInSession, artist, song, length)"
            query = query + "VALUES (%s, %s, %s, %s, %s)"           
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        print('Inserting COMPLETED!')
except Exception as e:
    print(e)

Inserting COMPLETED!


#### Query 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [78]:
query_select1 = "select artist, song, length from app_history where sessionId=338 and itemInSession=4;"
try:
    rows = session.execute(query_select1)
    df = pd.DataFrame(rows.current_rows, columns=['Artist', 'Song', 'Length'])
    df = df.rename(index = lambda x: x+1)
except Exception as e:
    print(e)

## Display the result
df.head()

,Artist,Song,Length
1,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### To satisfy the above request, we will need to create a table that contains, the artist column, song column, first name column, last name column and the userId, sessionId and itemInSession as their composite primary key. `userId` and `sessionId` are perfect for the composite partition key, but they also need the itemInSession column to uniquely identify each row.  Since the query requires the data to be sorted by the itemInSession column, it makes a good choice for the composite partition key.

In [79]:
query2 = """
CREATE TABLE IF NOT EXISTS user_playlist 
(userId INT, sessionId INT, itemInSession INT, artist TEXT, song TEXT, firstName TEXT, lastName TEXT, 
PRIMARY KEY((userId, sessionId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC);
"""
try:
    session.execute('DROP TABLE IF EXISTS user_playlist;')
    session.execute(query2)
    print('user_playlist CREATED!')
except Exception as e:
    print(e)
                    

user_playlist CREATED!


In [80]:
## Populate the table with data 
file = 'event_datafile_new.csv'

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            query = "INSERT INTO user_playlist (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
            query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
            
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        print('Inserting user_playlist COMPLETED!')
except Exception as e:
    print(e)

Inserting user_playlist COMPLETED!


#### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [81]:
query_select2 = "select artist, song, firstName, lastName FROM user_playlist WHERE userId = 10 AND sessionId = 182;"
try:
    rows = session.execute(query_select2)
    df = pd.DataFrame(rows.current_rows, columns=['Artist', 'Song', 'First Name', 'Last Name'])
    df = df.rename(index = lambda x: x+1)
except Exception as e:
    print(e)

## Display the result    
df.head()

,Artist,Song,First Name,Last Name
1,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
2,Three Drives,Greece 2000,Sylvie,Cruz
3,Sebastien Tellier,Kilometer,Sylvie,Cruz
4,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
#### For this query, we only need the first name, last name and song title columns in the table.  Since each columns on their own are not sufficient to act as the primary key of the table, we are combining them together as a composite partition key.

In [82]:
query3 = """
CREATE TABLE IF NOT EXISTS music_history 
(song TEXT, firstName TEXT, lastName TEXT,
PRIMARY KEY(song, firstName, lastName)) WITH CLUSTERING ORDER BY (firstName ASC, lastName ASC);
"""
try:
    session.execute('DROP TABLE IF EXISTS music_history;')
    session.execute(query3)
    print('music_history CREATED!')
except Exception as e:
    print(e)

music_history CREATED!


In [83]:
file = 'event_datafile_new.csv'

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            ## Prepare and insert data into the Cassandra table
            query = "INSERT INTO music_history (song, firstName, lastName)"
            query = query + "VALUES (%s, %s, %s)"
            
            session.execute(query, (line[9], line[1], line[4]))
        print('Inserting music_history COMPLETED!')
except Exception as e:
    print(e)
                    

Inserting music_history COMPLETED!


#### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [84]:
query_select3 = "select firstName, lastName from music_history where song = 'All Hands Against His Own';"
try:
    rows = session.execute(query_select3)
    df = pd.DataFrame(rows.current_rows, columns=['First Name', 'Last Name'])
    df = df.rename(index = lambda x: x+1)
except Exception as e:
    print(e)
    
## Display the result
df.head()

,First Name,Last Name
1,Jacqueline,Lynch
2,Sara,Johnson
3,Tegan,Levine


### Drop the tables before closing out the sessions

In [85]:
## Drop the table before closing out the sessions
try:
    rows = session.execute("drop table if exists app_history;")
    rows = session.execute("drop table if exists user_playlist;")
    rows = session.execute("drop table if exists music_history;")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [86]:
session.shutdown()
cluster.shutdown()